In [1]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Promotion Slide Duplicate\Promotion Replacement Function.ipynb"

## Parameters

In [2]:
promo_type = False

normalized = True
national = False 

display_share = False  # True if Available
feature_share = False

In [3]:
max_total_size = {
        'Tesco | System': 5.8,
        'Tesco | Disposables': 9.9}


custom_colors = [
    RGBColor(91, 159, 153),    # Darker teal
    RGBColor(131, 199, 193),   # Brighter medium teal
    RGBColor(168, 216, 212),   # Original light teal
    RGBColor(198, 236, 232),   # Very light teal
]
# custom_colors = [
#     RGBColor(111, 179, 173),  
#     RGBColor(121, 189, 183),  
#     RGBColor(168, 216, 212),  
#     RGBColor(178, 226, 222), 
# ]


In [4]:
client_manuf = ["Edgewell"]
client_brands = ["Billie","Hydro Silk","Intuition","Skintimate"]
decimals = 2
sign = "Before"
currency = '$' 
currency = ' '+ currency if sign.lower() == 'after' else  currency + ' '

categories = ["Manual Shave Women"]
sectors = ["Disposable","System"]
segments = ["Disposable","Hybrid","Razors","Refills"]
subsegments= ["1 Blade", "2 Blade", "3 Blade", "4 Blade", "5 Blade", "6 Blade","Rem Blades"]
subcategories= ["Dry Skin","Normal Skin","Rem Types","Sensitive Skin"]




customareas='REGIONS'
national = False
areas = [customareas]#customareas]#RETAILER"]#,customareas,"NATIONAL",]
 
regions_RET  = []#"CVS Corp","Walmart"]
channels_RET = []#"CVS","Walmart Div1 Corp","Walmart Nm Corp","Walmart Sc Corp"]
market_RET = []

regions_CHAN = []
channels_CHAN = []
market_CHAN = []
 
regions_CUST = []
channels_CUST = ["CVS Michigan","CVS Mid South","CVS Midwest","CVS Northeast","CVS South","CVS Southwest","CVS West","Walmart East","Walmart North","Walmart Southeast","Walmart Southwest","Walmart West"]
market_CUST = []

data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending August 2024"
years = ['2021', '2022','2023']



subcatg_parent = "Segment"
subcatg_parent_list = segments
percent = 100000
percentstr="'00 000"



# Add one month to the original ending date (YYYY-MM-01)
start_date = "2021-10-03"	
end_date = "2024-09-01"
prodORitem = "Product"


# Guidline Promo Columns ex :Volume Uplift >>> "[Measures].[Volume Uplift IYA]" using filter_dictionary_keys(fieldsNamePosition, 'Volume Upli')
#promo_col = ['[Measures].[Straight Discount 10-20 Sales]','[Measures].[Straight Discount 20-30 Sales]', '[Measures].[Straight Discount 30-40 Sales]','[Measures].[Straight Discount 40+ Sales]']
promo_col = []
selectedBrands = client_brands 
marketList = regions_RET + channels_RET + market_RET + regions_CHAN + channels_CHAN + market_CHAN 
notInScope = []
OpenEditData=True
direct_parent = {"Sector":"Category",
                "Segment":"Sector",
                "SubSegment":"Segment", 
                "SubCategory":"Segment"}

In [5]:

Scope = {
    "Category": categories,
    "Sector": sectors,
    "Segment": segments,
    "Subsegment": subsegments,
    "Subcategory": subcategories
}
suffixes = ["Category", "Sector", "Segment", "Subsegments", "Subcategory"]

In [6]:
def DetectHeader(df):
  # df = df.replace( np.nan, 0)
  vals=df.values
  vals=list(map(lambda x : all([type(i)==str for i in x ]),vals))
  # print(vals)
  break_point=0
  for i,v in enumerate(vals):
    if v:
      break_point=i
      # print(break_point)
      break  
  df.columns=df.iloc[break_point]
  df=df.iloc[break_point+1:,:]
  return df

## Reading datasets

In [7]:
loaded_data = {}
datasets_path = os.getcwd()+"/Promotion Datasets REGION/"
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        globals()[d.split('.')[0]] = pd.read_pickle(handle)

## Data Cleaning

In [8]:
def cleaningData(data):
    """
    Clean and preprocess data in a dictionary of DataFrames.

    Parameters:
    - data (dict): Dictionary containing DataFrames.

    Returns:
    - dict: Dictionary containing cleaned DataFrames.
    """
    cleaned_data = {}
    
    # Iterate over each key-value pair in the input dictionary
    for key in data:
        # Skip the first 11 rows if there are NaN values
        # df = data[key].iloc[cut_df-2:]
        # if data[key].iloc[cut_df-2,:].isna().any():
        #     df = data[key].iloc[cut_df-1:]
        df=DetectHeader(data[key])
        # Set column names and skip the first row
        if  df.columns.isna().any():
            continue
        # Perform specific cleaning operations based on the DataFrame columns and key
        if df.shape[0] > 0 and not 'National' in key:
            if 'Top Brands' in df.columns and 'Product' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                df['Product'].fillna('', inplace=True)
                df.fillna(0, inplace=True)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                df = df.reset_index(drop=True)
            
            elif 'Top Brands' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                #df.fillna(0, inplace=True)
                if normalized:
                    df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])] = df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift Normalized IYA'].isna(), None, df['Value Uplift Normalized IYA'] - 1)
                else:
                    df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])] = df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift IYA'].isna(), None, df['Value Uplift IYA'] - 1)
               
                df['VSOD Evaluation vs YA'] = np.where(df['VSOD IYA'].isna(), None, df['VSOD IYA'] - 1)

                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                df = df[~df['Top Brands'].str.contains('Total', case=False)]
                df = df[df['Total Size'] == 0].reset_index(drop=True)

       
                
            elif 'End of Week' in df.columns and 'Product' in df.columns:
                df['Product'] = df['Product'].fillna(method='ffill')
                df = df[(df['End of Week'].str.contains('2023|2024')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                new_start = "2023-07-31"
                df = df[(df['End of Week'] >= new_start) & (df['End of Week'] <= end_date)]
                df = df[~df['Product'].str.contains('Total', case=False)].reset_index(drop=True)
                df = df[df['Promo Sales'] > 1000]
                if normalized:
                    df = df.dropna(subset=['Value Uplift (v. base) Normalized'])
                    df =  df[df['Value Uplift (v. base) Normalized']<10]
                else:
                    df = df.dropna(subset=['Value Uplift (v. base)'])
                    df = df[df['Value Uplift (v. base']<10]
                df.fillna(0, inplace=True)
                df = df.reset_index(drop=True)
                
            elif 'End of Week' in df.columns:
                df['End of Week'] = df['End of Week'].astype(str)
                df = df[~df['End of Week'].str.contains('Total', case=False)].reset_index(drop=True)
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df['End of Week'] = df['End of Week'].dt.strftime("%d-%b-%y")
                df = df[(df['End of Week'].str.contains('-21|-22|-23|Jan-24')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                df = df.dropna()
                
            elif 'Grand Total' in df.columns:
                if 'Sector' == df.columns[1]:
                    df[direct_parent["Sector"]].fillna(method='ffill', inplace= True)
                    df['Sector'] = df['Sector'].replace(0, np.nan)
                    df['Sector'].fillna(method='ffill', inplace=True)
                    df['Sector'] = df.apply(lambda row: row[direct_parent["Sector"]] if 'Total' in row[direct_parent["Sector"]] else row['Sector'], axis=1)

                elif 'Segment' == df.columns[1]:
                    df['Segment'] = df['Segment'].replace(0, np.nan)  
                    df[direct_parent["Segment"]].fillna(method='ffill', inplace= True)          
                    df['Segment'] = df.apply(lambda row: row[direct_parent["Segment"]] if 'Total' in row[direct_parent["Segment"]] else row['Segment'], axis=1)
                    df['Segment'].fillna(method='ffill', inplace=True)
                elif 'SubSegment' == df.columns[1]:
                    df['SubSegment'] = df['SubSegment'].replace(0, np.nan)
                    df[direct_parent["SubSegment"]].fillna(method='ffill', inplace= True)          
                    df['SubSegment'] = df.apply(lambda row: row[direct_parent["SubSegment"]] if 'Total' in row[direct_parent["SubSegment"]] else row['SubSegment'], axis=1)
                    df['SubSegment'].fillna(method='ffill', inplace=True)
                elif 'SubCategory' == df.columns[1]:
                    df['SubCategory'] = df['SubCategory'].replace(0, np.nan)
                    df[direct_parent["SubCategory"]].fillna(method='ffill', inplace= True)          
                    df['SubCategory'] = df.apply(lambda row: row[direct_parent["SubCategory"]] if 'Total' in row[direct_parent["SubCategory"]] else row['SubCategory'], axis=1)
                    df['SubCategory'].fillna(method='ffill', inplace=True)
                df = df.reset_index(drop=True)
        df.fillna(0, inplace=True)
            # Check if the key matches specific categories and modify the key accordingly
        if key.split(' | ')[0] in categories and len(key.split(' | ')) == 3:
                modified_key = key.split(' | ')[1] + ' | ' + key.split(' | ')[2] + ' | ' + key.split(' | ')[0]
                if df.shape[0] > 0:
                    cleaned_data[modified_key] = df
        else:
                if df.shape[0] > 0:
                    cleaned_data[key] = df
    
    return cleaned_data


In [9]:
def cleaningdata_with_grand_total(data):
    """
    Clean and preprocess data in a dictionary of DataFrames.

    Parameters:
    - data (dict): Dictionary containing DataFrames.

    Returns:
    - dict: Dictionary containing cleaned DataFrames.
    """
    cleaningdata_with_grand_total = {}
    
    # Iterate over each key-value pair in the input dictionary
    for key in data:
        # Skip the first 11 rows if there are NaN values
        # df = data[key].iloc[cut_df-2:]
        # if data[key].iloc[cut_df-2,:].isna().any():
        #     df = data[key].iloc[cut_df-1:]
        df=DetectHeader(data[key])
        
        # # Set column names and skip the first row
        # df.columns = df.iloc[0]
        # df = df.iloc[1:]
        
        # Perform specific cleaning operations based on the DataFrame columns and key
        if df.shape[0] > 0 and not 'National' in key:
            if 'Top Brands' in df.columns and 'Product' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                df['Product'].fillna('', inplace=True)
                df.fillna(0, inplace=True)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
            
            elif 'Top Brands' in df.columns:
                df['Top Brands'] = df['Top Brands'].fillna(method='ffill')
                if normalized:
                    df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])] = df.loc[:,~ df.columns.isin(['VSOD IYA','Value Uplift Normalized IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift Normalized IYA'].isna(), None, df['Value Uplift Normalized IYA'] - 1)
                else:
                    df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])] = df.loc[:, df.columns.isin(['VSOD IYA','Value Uplift IYA'])].fillna(0)
                    df['Promo Value Uplift vs YA'] = np.where(df['Value Uplift IYA'].isna(), None, df['Value Uplift IYA'] - 1)
               
                df['VSOD Evaluation vs YA'] = np.where(df['VSOD IYA'].isna(), None, df['VSOD IYA'] - 1)
                df['Top Brands'] = df['Top Brands'].apply(lambda x: 'Grand Total' if 'Grand Total' in x else x.replace('Total', '').strip())
                #df = df[~df['Top Brands'].str.contains('Total', case=False)]
                df = df[df['Total Size'] == 0].reset_index(drop=True)

            elif 'End of Week' in df.columns and 'Product' in df.columns:
                df['Product'] = df['Product'].fillna(method='ffill')
                df = df[(df['End of Week'].str.contains('2023|2024')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                df = df[~df['Product'].str.contains('Total', case=False)].reset_index(drop=True)
                
                df = df[df['Promo Sales'] > 1000]
                if normalized:
                    df = df.dropna(subset=['Value Uplift (v. base) Normalized'])
                else:
                    df = df.dropna(subset=['Value Uplift (v. base)'])
                df.fillna(0, inplace=True)
                df = df.reset_index(drop=True)
                
            elif 'End of Week' in df.columns:
                df['End of Week'] = df['End of Week'].astype(str)
                df = df[~df['End of Week'].str.contains('Total', case=False)].reset_index(drop=True)
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df['End of Week'] = df['End of Week'].dt.strftime("%d-%b-%y")
                df = df[(df['End of Week'].str.contains('-21|-22|-23|Jan-24')) & (df['End of Week'].notna())]
                df['End of Week'] = pd.to_datetime(df['End of Week'])
                df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)]
                df = df.dropna()
                
            elif 'Grand Total' in df.columns:
                df['Sector'].fillna(method='ffill', inplace=True)
                df.fillna(0, inplace=True)
            
            # Check if the key matches specific categories and modify the key accordingly
            if key.split(' | ')[0] in categories and len(key.split(' | ')) == 3:
                modified_key = key.split(' | ')[1] + ' | ' + key.split(' | ')[2] + ' | ' + key.split(' | ')[0]
                if df.shape[0] > 0:
                    cleaningdata_with_grand_total[modified_key] = df
            else:
                if df.shape[0] > 0:
                    cleaningdata_with_grand_total[key] = df
    
    return cleaningdata_with_grand_total


In [10]:
def cleaning13New(data):
    """
    Clean and process data for specific brands and regions.

    Parameters:
    - data (dict): Dictionary containing raw data.

    Returns:
    - dict: Dictionary containing cleaned and processed data.
    """
    data_cleaned = {}
    
    # Define maximum total size for each combination of product type and region
    
    for key, df in data.items():
        # Skip processing if the region is 'NATIONAL' or 'National'
        if 'NATIONAL' in areas or 'National' in key:
            continue
        new_data = []
        # Skip first 12 rows as they are headers and metadata
        # df = df.iloc[12:]
        df=DetectHeader(data[key])
        
        # # Set columns names based on the first row, and skip the first row
        # df.columns = df.iloc[0]
        # df = df.iloc[1:]
        # Fill missing values in 'Top Brands' column with the previous non-null value
        df['Top Brands'].fillna(method='ffill', inplace=True)
        # Filter out rows where 'Top Brands' is 'Grand Total' or 'Other'
        df = df[(df['Top Brands'] != 'Grand Total') & (df['Top Brands'] != 'Other')]
        # Remove 'GR' suffix from 'Total Size' and convert it to integer
        df['Total Size'] = df['Total Size'].str.extract('(\d+)', expand=False)
        df.fillna('0',inplace=True)
        df['Total Size'] = df['Total Size'].astype(int)
        # Sort data by 'Value Share' in descending order
        df = df.sort_values(by='Value Share', ascending=False).reset_index(drop=True)
        for i, brand in enumerate(df['Top Brands'].unique()):
            # Determine the product key based on the first two elements of the key
            product_key = key.split('|')[0] + '|' + key.split('|')[1]
            # Get the maximum total size for the product key, if it exists
            max_size = max_total_size.get(product_key, None)
            # Filter rows for the current brand and check if total size is within the maximum allowed size
            if max_size is not None:
                brand_df = df[(df['Top Brands'] == brand) & (df['Total Size'] <= max_size)]
            else:
                brand_df = pd.DataFrame()
            # Calculate recruitment ratio if the brand has data and total size is greater than zero
            #brand_total = df[(df['Top Brands'] == brand + ' Total')]['Promo Value'].values
            brand_total = df[(df['Top Brands'].str.strip() == (brand + ' Total').strip())]['Promo Value'].values

            if not brand_df.empty and brand_total.size > 0 and brand_total[0] > 0:
                brand_sum = brand_df['Promo Value'].sum() / brand_total[0]
                new_data.append({'Top Brands': brand, 'Recruitment': brand_sum, 'Consumption': 1 - brand_sum, 'Value Share': df['Value Share'][i], 'SUM':brand_df['Promo Value'].sum()})
        # Create a new DataFrame with cleaned data
        new = pd.DataFrame(new_data)
        new.fillna(0, inplace=True)
        
        # Add cleaned data to the dictionary if it contains non-zero rows
        if new.shape[0] != 0:
            data_cleaned[key] = new
        
    return data_cleaned


In [11]:
def VSOD_Clean(VSOD_Data):
    """
    Clean and preprocess VSOD data in a dictionary of DataFrames.

    Parameters:
    - VSOD_Data (dict): Dictionary containing VSOD DataFrames.

    Returns:
    - dict: Dictionary containing cleaned VSOD DataFrames.
    """
    VSOD_cleaned = {}
    for key in VSOD_Data:
        # Skip first 11 rows
        # df = VSOD_Data[key].iloc[cut_df-2:]
        # # Set column names and skip first row
        # df.columns = df.iloc[0]
        # df = df.iloc[1:]
        df=DetectHeader(VSOD_Data[key])
        # Fill NaN values with 0
        # print(key)
        if 'Sector' == df.columns[1]:
            df[direct_parent["Sector"]] = df[direct_parent["Sector"]].replace(0, np.nan)
            df[direct_parent["Sector"]].fillna(method='ffill', inplace = True)
            df['Sector'] = df['Sector'].replace(0, np.nan)
            df['Sector'] = df.apply(lambda row: row[direct_parent["Sector"]] if 'Total' in row[direct_parent["Sector"]] else row['Sector'], axis=1)
            df=df[~df[direct_parent["Sector"]].str.contains(r'\btotal\b', case=False) | df[direct_parent["Sector"]].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['Sector'].fillna(method='ffill', inplace=True)
            
        elif 'Segment' == df.columns[1]:
            df[direct_parent["Segment"]].fillna(method='ffill', inplace=True)
            df['Segment'] = df['Segment'].replace(0, np.nan)
            # df=df[~df['Sector'].str.contains(r'\btotal\b', case=False) | df['Sector'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['Segment'] = df.apply(lambda row: row[direct_parent["Segment"]] if 'Total' in row[direct_parent["Segment"]] else row['Segment'], axis=1)
            df['Segment'].fillna(method='ffill', inplace=True)    
        elif 'SubSegment' == df.columns[1]:
            df['SubSegment'] = df['SubSegment'].replace(0, np.nan)
            df[direct_parent["SubSegment"]].fillna(method='ffill', inplace=True)
            # df=df[~df['Segment'].str.contains(r'\btotal\b', case=False) | df['Segment'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubSegment'] = df.apply(lambda row: row[direct_parent["SubSegment"]] if 'Total' in row[direct_parent["SubSegment"]] else row['SubSegment'], axis=1)
            df['SubSegment'].fillna(method='ffill', inplace=True)
        elif 'SubCategory' == df.columns[1]:
            df['SubCategory'] = df['SubCategory'].replace(0, np.nan)
            df[direct_parent["SubCategory"]].fillna(method='ffill', inplace=True)
            # df=df[~df['Segment'].str.contains(r'\btotal\b', case=False) | df['Segment'].str.contains(r'\bGrand\b', case=False)].reset_index(drop=True)
            df['SubCategory'] = df.apply(lambda row: row[direct_parent["SubCategory"]] if 'Total' in row[direct_parent["SubCategory"]] else row['SubCategory'], axis=1)
            df['SubCategory'].fillna(method='ffill', inplace=True)
        df.fillna(0, inplace=True)
        df = df.reset_index(drop=True)
        if df.shape[0] > 0:
            VSOD_cleaned[key] = df
    return VSOD_cleaned


In [12]:
def merging(VSODClient_Cleaned, VSODCleaned, col):
    """
    Merge two dictionaries of DataFrames based on a common column.

    Parameters:
    - VSODClient_Cleaned (dict): Dictionary containing cleaned VSOD client DataFrames.
    - VSODCleaned (dict): Dictionary containing cleaned VSOD DataFrames.

    Returns:
    - dict: Dictionary containing merged DataFrames.
    """
    merged_dict = {}
    for key in VSODClient_Cleaned:
        # Merge DataFrames based on 'Sector' column
        #merged_df = pd.merge(VSODCleaned[key], VSODClient_Cleaned[key], on=col, how='left')
        merged_df = pd.merge(VSODClient_Cleaned[key],VSODCleaned[key], on=col, how='left')
        #merged_df = merged_df.dropna(subset=['Grand Total'])
        merged_df['Grand Total'] = merged_df['Grand Total'].fillna(0)
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            merged_dict[key] = merged_df     
    return merged_dict


In [13]:
#modified_promotionBrandsP12M_display = cleaningData(display_data)

In [14]:
modified_promotionBrandsP12M = cleaningData(promotions_brands_P12M)

In [15]:
modified_promotionBrandsP12M_total = cleaningdata_with_grand_total(promotions_brands_P12M)

In [16]:
modified_promotionProductsP12M = cleaningData(promotions_products_P12M)

In [17]:
modified_promotionProductsP12M_updated = {}
for key, df in modified_promotionProductsP12M.items():
    df = df.copy()
    df = df[df['Product'] != '']
    df = df[df['Promo Sales'] >= 10000]
    df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
    if not df.empty:
        modified_promotionProductsP12M_updated[key] = df
modified_promotionProductsP12M_volumeuplift = modified_promotionProductsP12M_updated

In [18]:
modified_promotionEndOfWeek = cleaningData(promotions_EndOfWeek)

In [19]:
modified_valueUplift = cleaningData(value_uplift)

In [20]:
def cleaning13New(data):
    """
    Clean and process data for specific brands and regions.

    Parameters:
    - data (dict): Dictionary containing raw data.

    Returns:
    - dict: Dictionary containing cleaned and processed data.
    """
    data_cleaned = {}
    
    # Define maximum total size for each combination of product type and region
    
    for key, df in data.items():
        # Skip processing if the region is 'NATIONAL' or 'National'
        if 'National' in key:
           continue
        
        new_data = []
        df=DetectHeader(data[key])

        # Skip first 12 rows as they are headers and metadata
        # df = df.iloc[cut_df-1:]
        
        # # Set columns names based on the first row, and skip the first row
        # df.columns = df.iloc[0]
        # df = df.iloc[1:]
        # Fill missing values in 'Top Brands' column with the previous non-null value
        df['Top Brands'].fillna(method='ffill', inplace=True)
        # Filter out rows where 'Top Brands' is 'Grand Total' or 'Other'
        df = df[(df['Top Brands'] != 'Grand Total') & (df['Top Brands'] != 'Other')]
        # Remove 'GR' suffix from 'Total Size' and convert it to integer
        df['Total Size'] = df['Total Size'].str.extract('(\d+)', expand=False)
        df.fillna('0',inplace=True)
        df['Total Size'] = df['Total Size'].astype(int)
        df['Promo Value'] = df['Promo Value'].astype(int)
        
        # Sort data by 'Value Share' in descending order
        df = df.sort_values(by='Value Share', ascending=False).reset_index(drop=True)
        for i, brand in enumerate(df['Top Brands'].unique()):
            # Determine the product key based on the first two elements of the key
            product_key = key.split('|')[0] + '|' + key.split('|')[1]
            
            # Get the maximum total size for the product key, if it exists
            max_size = max_total_size.get(product_key, None)
            # Filter rows for the current brand and check if total size is within the maximum allowed size
            if max_size is not None:
                brand_df = df[(df['Top Brands'] == brand) & (df['Total Size'] <= max_size)]
            else:
                brand_df = pd.DataFrame()
                
            # Calculate recruitment ratio if the brand has data and total size is greater than zero
            #brand_total = df[(df['Top Brands'] == brand + ' Total')]['Promo Value'].values
            brand_total = df[(df['Top Brands'].str.strip() == (brand + ' Total').strip())]['Promo Value'].values

            if not brand_df.empty and brand_total.size > 0 and brand_total[0] > 0:
                brand_sum = brand_df['Promo Value'].sum() / brand_total[0]
                new_data.append({'Top Brands': brand, 'Recruitment': brand_sum, 'Consumption': 1 - brand_sum, 'Value Share': df['Value Share'][i], 'SUM':brand_df['Promo Value'].sum()})
        
        # Create a new DataFrame with cleaned data
        new = pd.DataFrame(new_data)
        new.fillna(0, inplace=True)
        # Add cleaned data to the dictionary if it contains non-zero rows
        if new.shape[0] != 0:
            data_cleaned[key] = new
        
    return data_cleaned


In [21]:
newModifiedBrands = cleaning13New(promotions_brands_P12M)

In [22]:
if len(sectors) >0:
    a = VSOD_Clean(Sector_VSOD)
    if len(Sector_client_VSOD) >0:
        b = cleaningData(Sector_client_VSOD)
        sect_vsod_merged = merging(b,a, col=[direct_parent["Sector"],'Sector'])
    else:
        sect_vsod_merged = a
    c = cleaningData(Sector_manuf_VSOD)
    for key in sect_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(sect_vsod_merged[key], c[key], on=[direct_parent["Sector"],'Sector'], how='left')
        if merged_df.shape[0]>0:
            sect_vsod_merged[key] = merged_df    

if len(segments) >0:
    a = VSOD_Clean(Segment_VSOD)
    if len(Segment_client_VSOD) > 0:
        b = cleaningData(Segment_client_VSOD)
        seg_vsod_merged = merging(a,b, col=[direct_parent["Segment"],'Segment'])
    else:
        seg_vsod_merged = a
    
    c = cleaningData(Segment_manuf_VSOD)
    for key in seg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(seg_vsod_merged[key], c[key], on=[direct_parent["Segment"],'Segment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            seg_vsod_merged[key] = merged_df    

if len(subsegments) >0:
    a = VSOD_Clean(SubSegment_VSOD)
    if len(SubSegment_client_VSOD) > 0 :
        b = cleaningData(SubSegment_client_VSOD)
        subseg_vsod_merged = merging(a,b, col=[direct_parent["SubSegment"],'SubSegment'])
    else:
        subseg_vsod_merged = a
    c = cleaningData(SubSegment_manuf_VSOD)
    for key in subseg_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subseg_vsod_merged[key], c[key], on=[direct_parent["SubSegment"],'SubSegment'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subseg_vsod_merged[key] = merged_df    

if len(subcategories) >0:
    a = VSOD_Clean(SubCategory_VSOD)
    if len(SubCategory_client_VSOD) > 0 :
        b = cleaningData(SubCategory_client_VSOD)
        subcat_vsod_merged = merging(a,b, col=[direct_parent["SubCategory"],'SubCategory'])
    else:
        subcat_vsod_merged = a
    c = cleaningData(SubCategory_manuf_VSOD)
    for key in subcat_vsod_merged:
        # Merge DataFrames based on 'Sector' column
        merged_df = pd.merge(subcat_vsod_merged[key], c[key], on=[direct_parent["SubCategory"],'SubCategory'], how='left')
        merged_df = merged_df.fillna(0)
        if merged_df.shape[0]>0:
            subcat_vsod_merged[key] = merged_df  

In [23]:
def splitkeys(dic, lis, parent,clientlist):
    """
    Splits the keys of a dictionary into new keys based on unique values in a specified column.
    Parameters:
    dic (dict): The input dictionary with DataFrames as values.
    lis (list): A list of sector names to filter by (if needed).
    parent (str): The column name used for splitting (e.g., 'Sector').
    
    Returns:
    dict: A new dictionary with updated keys and filtered DataFrames.
    """
    splitvsod = {}
    for key in dic.keys():
        for key in dic.keys():
            # Split the key into parts and check if all parts are in the valid list
    
                # Get a copy of the current DataFrame
            s = dic[key].copy()        
            for value in s[parent].unique():
                if isinstance(value, str) and not value.endswith("Total"):
                    new_key = f"{key} | {value}"                
                    filtered_df = s[s[parent].isin([value, f"{value} Total"])]   
                    for cli in clientlist:
                        needed_col = [filtered_df.columns[0],filtered_df.columns[1],"VSOD",cli] 
                        
                        filtered_dfnew = filtered_df[needed_col]
                        
                        splitvsod[new_key + " | " + cli] = filtered_dfnew
    keys_to_remove = [
        k for k in splitvsod.keys() 
        if k.split(" | ")[-2] not in lis
    ]
    for k in keys_to_remove:
        del splitvsod[k]
    return splitvsod


In [24]:
client=client_brands+client_manuf
sect_vsod_merged=splitkeys(sect_vsod_merged,categories,parent=direct_parent['Sector'],clientlist=client)
seg_vsod_merged=splitkeys(seg_vsod_merged,sectors,parent=direct_parent['Segment'],clientlist=client)
subseg_vsod_merged=splitkeys(subseg_vsod_merged,segments,parent=direct_parent['SubSegment'],clientlist=client)
subcat_vsod_merged=splitkeys(subcat_vsod_merged,segments,parent=direct_parent['SubCategory'],clientlist=client)

In [25]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    #seg_vsod_count = seg_vsod_count * len(sectors) 
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    #subseg_vsod_count =subseg_vsod_count *len(segments)
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    #subcat_vsod_count = subcat_vsod_count * len(subsegments)
    subcat_vsod_count = subcat_vsod_count

 

In [26]:
promotionsBrandSortedTotalFinal={}
promotionsBrandSortedTotal=dfSort(modified_promotionBrandsP12M, client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in promotionsBrandSortedTotal.items():
     df_client = selectClientBrands(promotionsBrandSortedTotal[key],'Top Brands', 'Promo Value')
     number_of_brands_needed = max(6 - len(df_client),0)
     df = df[~df['Top Brands'].isin(client_brands)]
     df = df.sort_values(by='Promo Value', ascending=False).head(number_of_brands_needed)
     df = pd.concat([df, df_client], ignore_index=True)
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
        
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     
     if df.shape[0] >0:
          promotionsBrandSortedTotalFinal[key] = df

In [27]:
promotionsBrandNOTSortedTotalFinal={}
promotionsBrandNOTSortedTotalFinal=dfSort(modified_promotionBrandsP12M, client_brands, "Top Brands", num=8,salesCol='Promo Value')
for key,df in modified_promotionBrandsP12M.items():
     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
     df = df[~df['Top Brands'].str.contains('Others', case=False)]
     df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
     df = df[df['Value Share'] > 0.01]
     df['VSOD Evaluation vs YA'] = df['VSOD Evaluation vs YA'].astype(float)
     df['Promo Value Uplift vs YA'] = df['Promo Value Uplift vs YA'].astype(float)
     if df.shape[0] >0:
          promotionsBrandNOTSortedTotalFinal[key] = df

In [28]:
selectedBrands_og = selectedBrands
selectedBrands= selectedBrands + ["Grand Total"]

In [29]:
promotionsBrandsSelected={key:modified_promotionBrandsP12M_total[key][modified_promotionBrandsP12M_total[key]['Top Brands'].isin(selectedBrands)].sort_values(by='Promo Value',ascending=False) for key in modified_promotionBrandsP12M_total.keys()   if all(cat != key.split(' | ')[0] for cat in categories)}

In [30]:
for key in promotionsBrandsSelected:
    # Identify the Grand Total row (adjust this if necessary to match your data)
    grand_total_row = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] == 'Grand Total']
    # Remove the Grand Total row from the dataframe
    sorted_df = promotionsBrandsSelected[key].loc[promotionsBrandsSelected[key]['Top Brands'] != 'Grand Total']
    # Concatenate the Grand Total row to the top of the dataframe
    promotionsBrandsSelected[key] = pd.concat([grand_total_row, sorted_df], ignore_index=True)

In [31]:
selectedBrands = selectedBrands_og

In [32]:
# Not including client brands
promotionsNotBrandsSelected = {
    key: modified_promotionBrandsP12M_total[key][
        ~modified_promotionBrandsP12M_total[key]['Top Brands'].isin(selectedBrands)
    ].sort_values(by='Value Share', ascending=False)
    for key in modified_promotionBrandsP12M_total.keys()
    if all(cat != key.split(' | ')[0] for cat in categories)
}


In [33]:
def concatAttribute(dic, marketList):
    """
    This function takes a dictionary of DataFrames and a list of markets, and concatenates
    the DataFrames by adding a 'SOURCE' column to each DataFrame.

    Parameters:
    dic (dict): A dictionary where keys are strings in the format 'market | source', and
                values are DataFrames containing market data.
    marketList (list): A list of market names (strings).

    Returns:
    dict: A dictionary where keys are market names and values are concatenated DataFrames
          with an added 'SOURCE' column.
    """
    # Initialize a defaultdict to store the resulting DataFrames
    marketDic = defaultdict(list)
    
    # Iterate through the list of markets
    for market in marketList:
        # Iterate through the items in the dictionary
        for key, value in dic.items():
            # Check if the market name matches the key's market part
            if market == key.split(' | ')[0]:
                # Extract the source part from the key and assign it to the 'SOURCE' column
                value['SOURCE'] = list(set(key.split(' | ')) - set([market]))[0]
                value = value[value['Value Share'] >0.01]
                value = value[~value['Top Brands'].str.contains('Other')].reset_index(drop=True)
            
                # Only include rows where 'SOURCE' is not 'National'
                if (value['SOURCE'] != 'National').all():
                    marketDic[market].append(value)

        # Concatenate all DataFrames in the list for each market
        if len(marketDic[market]) != 0:
            marketDic[market] = pd.concat(marketDic[market])
    
    return marketDic

def fillingMissingBrands(dic):
    """
    This function fills in missing brands for each market and source combination in the
    provided dictionary of DataFrames.

    Parameters:
    dic (dict): A dictionary where keys are market names and values are DataFrames
                containing market data with 'Top Brands' and 'SOURCE' columns.

    Returns:
    dict: The input dictionary with missing brands filled in each DataFrame.
    """
    # Iterate through the dictionary items
    for key, value in dic.items():
        # Get the unique list of top brands in the DataFrame
        brandList = value['Top Brands'].unique().tolist()
        # Iterate through the unique sources in the DataFrame
        for source in value['SOURCE'].unique():
            # Check if the number of unique brands for the source is less than the total unique brands
            if value[value['SOURCE'] == source]['Top Brands'].nunique() != len(brandList):
                # Find the missing brands for the source
                missingBrand = list(set(brandList) - set(value[value['SOURCE'] == source]['Top Brands'].unique()))
                # Create a DataFrame for the missing brands with the current source
                missingBrand = pd.DataFrame({'Top Brands': missingBrand, 'SOURCE': source}).explode('Top Brands')
                # Concatenate the missing brands DataFrame with the original DataFrame
                value = pd.concat([value, missingBrand]).replace(np.nan, 0).reset_index(drop=True)
        # Update the dictionary with the filled DataFrame
        dic[key] = value
    
    return dic


In [34]:
promotionsBrandsWithMarket=concatAttribute(promotionsBrandsSelected,marketList)
promotionsBrandsWithMarket = fillingMissingBrands(promotionsBrandsWithMarket)


In [35]:
promotionsNotBrandsWithMarket=concatAttribute(promotionsNotBrandsSelected,marketList)
promotionsNotBrandsWithMarket = fillingMissingBrands(promotionsNotBrandsWithMarket)


In [36]:
def split_market(data, Scope):
    final = {}
    for k,df in data.items():
        for key, value in Scope.items():
            df_market = df[df['SOURCE'].isin(value)]
            df_market = df_market.reset_index(drop=True)
            if df_market.shape[0] >0:
                final[k + ' | ' + value[0]] = df_market
    return final        

In [37]:
newpromotionsBrandsWithMarket = split_market(promotionsBrandsWithMarket,Scope)
newpromotionsNotBrandsWithMarket = split_market(promotionsNotBrandsWithMarket,Scope)

In [38]:
def concatAttributeNew(sorted):
    """
    This function takes a dictionary of DataFrames sorted by keys and concatenates the DataFrames
    based on specified categories, sectors, and segments. It adds a 'SOURCE' column to each DataFrame.

    Parameters:
    sorted (dict): A dictionary where keys are strings in the format 'category | sector | segment | brand',
                   and values are DataFrames containing market data.

    Returns:
    dict: A dictionary where keys are the categories, sectors, and segments, and values are concatenated DataFrames
          with an added 'SOURCE' column.
    """
    # List of all categories, sectors, and segments to process
    lis = categories + sectors + segments + subsegments + subcategories
    #Extra this is because edgewell has something wrong in segments
    lis =  lis[:-1]
    # Initialize a defaultdict to store the resulting DataFrames
    marketDic = defaultdict(list)
    concatenatedDic = {}
    # Iterate through the list of categories, sectors, and segments
    for i in lis:
        # Iterate through the items in the sorted dictionary
        for key, value in sorted.items():
            # Check if the current item is part of the key
            if i in key:
                # Split the key to extract individual parts              
                parts = key.split(' | ')
                # Identify the client brand (either 'Extra' or 'Kiwi')
                if i in categories:
                    markets = parts[1]
                else:
                    markets = parts[0]
                
                # Set the 'SOURCE' column to the client brand
                value['SOURCE'] = markets
                marketDic[i].append(value)
        # Concatenate all DataFrames in the list for each category/sector/segment
        if marketDic[i]:
            concatenatedDic[i] = pd.concat(marketDic[i], ignore_index=True)
    return concatenatedDic


In [39]:
concated = concatAttributeNew(modified_promotionBrandsP12M)

In [40]:
def filter_data(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        df = modified_promotionProductsP12M[key]
        df = df[df['Product'] != '']
        df=df.sort_values(by=['Promo Share'], ascending=False)
        # Filter and sort the DataFrame
        df['cumulative promo share'] = df['Promo Share'].cumsum()
        df = df[df['Discount Depth (%)'] >= 0.05]
        df = df[df['VSOD'] >= 0.05]
        df = df[df['cumulative promo share'] <= 0.8]
        df = df.sort_values(by='Incr Value', ascending=False).reset_index(drop=True)
        df = df.head(50)
        df['index'] = str(df.index + 1)
        df = df.reset_index(drop=True)
        if df.shape[0] >0:
            cleaned_data[key] = df
        #else:
            #print(key)
    return cleaned_data
        

In [41]:
new_modified_promotionProductsP12M = filter_data(modified_promotionProductsP12M)

In [42]:
def filter_data_Top(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        combined_df = pd.DataFrame() 
        for client in client_brands:
            
            df = modified_promotionProductsP12M[key]
            # Filter the DataFrame for the current client brand
            df = df[df['Product'] != '']
            df = df[df['Top Brands'] == client]
            df = df.sort_values(by='Top Brands')
            df['Promo Share'] = pd.to_numeric(df['Promo Share'], errors='coerce')
            df['cumulative promo share'] = df.groupby('Top Brands')['Promo Share'].cumsum()
            df = df[df['cumulative promo share'] <= 0.8]

            df = df[df['Discount Depth (%)'] >= 0.05]
            df = df[df['VSOD'] >= 0.05]
            if df.shape[0] >0:
                combined_df = pd.concat([combined_df, df])
                combined_df = combined_df.sort_values(by='Incr Value', ascending=False).head(20).reset_index(drop=True)
        if combined_df.shape[0] > 0:
            cleaned_data[key] = combined_df.reset_index(drop=True)  # Store the combined DataFrame for the current key
                
    return cleaned_data
        

In [43]:
# modified_promotionProductsP12M_updated = {}
# for key, df in modified_promotionProductsP12M.items():
#     df = df.copy()
#     df = df[df['Product'] != '']
#     df = df[df['Promo Sales'] >= 10000]
#     df = df.sort_values(by='Promo Value', ascending=False).reset_index(drop=True)
#     if not df.empty:
#         modified_promotionProductsP12M_updated[key] = df
# modified_promotionProductsP12M = modified_promotionProductsP12M_updated

In [44]:
def filter_data_Bot(modified_promotionProductsP12M):
    cleaned_data = {}
    for key in modified_promotionProductsP12M:
        combined_df = pd.DataFrame() 
        for client in client_brands:
            df = modified_promotionProductsP12M[key]
            # Filter the DataFrame for the current client brand
            df = df[df['Product'] != '']
            df = df[df['Top Brands'] == client]
            df = df.sort_values(by='Top Brands')
            df['Promo Share'] = pd.to_numeric(df['Promo Share'], errors='coerce')
            df['cumulative promo share'] = df.groupby('Top Brands')['Promo Share'].cumsum()
            df = df[df['Discount Depth (%)'] >= 0.05]
            df = df[df['VSOD'] >= 0.05]
            if df.shape[0] >0:
                combined_df = pd.concat([combined_df, df])
                combined_df = combined_df.sort_values(by='Incr Value', ascending=False).tail(20).reset_index(drop=True)
                combined_df = combined_df.sort_values(by ='Incr Value', ascending= True).reset_index(drop=True)
        if combined_df.shape[0] > 0:
            cleaned_data[key] = combined_df.reset_index(drop=True)  # Store the combined DataFrame for the current key
    return cleaned_data
        

In [45]:
top20clientonly = filter_data_Top(modified_promotionProductsP12M)

bottom20clientonly = filter_data_Bot(modified_promotionProductsP12M)

In [46]:
def promotionsEndOfWeekCleaning(promotions_EndOfWeek, notInScope, col='Top Brands'):
    """
    Clean promotions end of week data.

    Parameters:
    promotions_EndOfWeek (dict): Dictionary containing promotions end of week data.
    notInScope (list): List of items not in scope to be excluded.
    col (str): Column name to check for filtering. Default is 'Top Brands'.

    Returns:
    dict: Dictionary containing cleaned promotions end of week data.
    """
    # Initialize an empty dictionary to store cleaned promotions end of week data
    promotionsEndOfWeek = {}
    # Iterate over items in promotions_EndOfWeek dictionary
    for key, value in promotions_EndOfWeek.items():
        # Make a copy of the dataframe to avoid modifying the original data
        df = value.copy()
        # Check if the dataframe is not empty
        if df.shape[0] != 0:
            # Modify the key to match the desired format if applicable
            modified_key = key
            if key.split(' | ')[0] in categories and len(key.split(' | ')) == 3:
                modified_key = key.split(' | ')[1] + ' | ' + key.split(' | ')[2] + ' | ' + key.split(' | ')[0]
            # Check if the key contains any item from the notInScope list
            # If not, add the dataframe to the cleaned dictionary after filtering out 'Grand Total' rows
            flag = False if any(element in modified_key for element in notInScope) else True
            if flag:
                promotionsEndOfWeek[modified_key] = df[df[col] != 'Grand Total'].reset_index(drop=True).replace(np.nan, 0)
        else:
            print(key, ' Is empty')
    
    return promotionsEndOfWeek


In [47]:
mod=cleaningData(promotions_EndOfWeek)
promotionsEndOfWeekCleaned=promotionsEndOfWeekCleaning(mod,notInScope,col='End of Week')


In [48]:
brandMarket = list(set([key.split(' | ')[0]+' | '+key.split(' | ')[1] for key in promotionsEndOfWeekCleaned]))
brandMarketCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat in key.split(' | ')[-1] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[-1] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[-1] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[-1] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in promotionsEndOfWeekCleaned.keys() if any(cat == key.split(' | ')[-1] for cat in subcategories )]


In [49]:
def completeDates(dfList, promotionsEndOfWeekCleaned):
    """
    Complete dates for each dataframe in dfList based on promotionsEndOfWeekCleaned.

    Parameters:
    dfList (list): List of dataframe keys.
    promotionsEndOfWeekCleaned (dict): Dictionary containing cleaned promotions end of week data.

    Returns:
    tuple: Tuple containing EndOfWeekcompletDate (dictionary), dfGroup (list), and dic (dictionary).
    """
    # Create a list of unique brand-category combinations
    brandCatList = sorted(set(key.split(' | ')[0] + ' | ' + key.split(' | ')[2] for key in dfList))
    
    # Initialize dictionaries and lists
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    
    # Count occurrences of each brand-category combination
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]) and (key.split(' | ')[1] == name.split(' | ')[2]):
                dic[key] += 1
                
    # Iterate over unique brand-category combinations
    for name in dic.keys():
        # Get dataframe keys associated with the current brand-category combination
        dfName = [key for key in dfList if name == (key.split(' | ')[0] + ' | ' + key.split(' | ')[2])]
        
        # Extract unique dates from all associated dataframes
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['End of Week']].drop_duplicates()
        if uniqueDates.shape[0] > 0:
            # Initialize dictionary for complete dates for each dataframe
            dfCompleteDates = {}
            
            # Add dataframe keys to the group list
            dfGroup.append(dfName)
            
            # Populate EndOfWeekcompletDate dictionary with dataframes merged on unique dates
            for key in dfName:
                EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left').replace(np.nan, 0).sort_values(by='End of Week').reset_index(drop = True)
    return EndOfWeekcompletDate, dfGroup, dic


In [50]:
if len(categories) != 0:
    dfCategory,catGroup,catDuplication=completeDates(brandMarketCategory,promotionsEndOfWeekCleaned)
if len(sectors) != 0:
    dfSector,secGroup,secDuplication=completeDates(brandMarketSector,promotionsEndOfWeekCleaned)
if len(segments) != 0:
    dfSegment,segGroup,segDuplication=completeDates(brandMarketSegment,promotionsEndOfWeekCleaned)
if len(subsegments) != 0:
    dfSubSegment,subsegGroup,subsegDuplication=completeDates(brandMarketSubSegment,promotionsEndOfWeekCleaned)
if len(subcategories) != 0:
    dfSubCategory,subcatGroup,subcatDuplication=completeDates(brandMarketSubCategory,promotionsEndOfWeekCleaned)

In [51]:
if len(sectors):
    sect_vsod_count =0
    for key,df in sect_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                sect_vsod_count +=1
    sect_vsod_count = sect_vsod_count *len(categories)
 
if len(segments):
    seg_vsod_count =0
    for key,df in seg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                seg_vsod_count +=1
    #seg_vsod_count = seg_vsod_count * len(sectors) 
    seg_vsod_count = seg_vsod_count           
 
if len(subsegments) >0:
    subseg_vsod_count =0
    for key,df in subseg_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subseg_vsod_count +=1
    #subseg_vsod_count =subseg_vsod_count *len(segments)
    subseg_vsod_count = subseg_vsod_count
 
if len(subcategories) >0:
    subcat_vsod_count =0
    for key,df in subcat_vsod_merged.items():
        client_manuf_brands = client_brands + client_manuf
        for client in client_manuf_brands:
            if client in df.columns:
                subcat_vsod_count +=1
    #subcat_vsod_count = subcat_vsod_count * len(subsegments)
    subcat_vsod_count = subcat_vsod_count

In [52]:
PromoRet ={}
if len(categories)!=0:
    first_key, first_value = next(iter(catDuplication.items()))
    PromoRet.update({first_key: first_value})
if len(sectors)!=0:
    sec_key, sec_value = next(iter(secDuplication.items()))
    PromoRet.update({sec_key:sec_value})
if len(segments)!=0:
    third_key, third_value = next(iter(segDuplication.items()))
    PromoRet.update({third_key: third_value})
if len(subsegments)!=0:
    fourth_key, fourth_value = next(iter(subsegDuplication.items()))
    PromoRet.update({fourth_key:fourth_value})
if len(subcategories)!=0:
    fifth_key, fifth_value = next(iter(subcatDuplication.items()))
    PromoRet.update({fifth_key:fifth_value })

In [53]:
PromoSalesTypes_data = {}
for key, df in modified_promotionBrandsP12M.items():
    df = df[~df['Top Brands'].str.contains('Others', case=False)]
    df = df[~df['Top Brands'].str.contains('Grand Total', case=False)]
    df = df[df['Value Share'] > 0.01]
    df = df[df['Promo Value'] > 0]
    # Select client brands and additional brands needed to make 10 brands
    df_client = selectClientBrands(modified_promotionBrandsP12M[key],'Top Brands', 'Value Share')
    number_of_brands_needed = max(10 - len(df_client),0)
    df = df[~df['Top Brands'].isin(client_brands)]
    df = df.sort_values(by='Value Share', ascending=False).head(number_of_brands_needed)
    # Concatenate client brands and additional brands
    df = pd.concat([df_client, df], ignore_index=True)
    df = df[df['Promo Value'] > 0]
    df = df.reset_index(drop=True)
    if df.shape[0]:
        PromoSalesTypes_data[key] =df

In [54]:
final_lis = []
cat_lis = []
if categories:
    for i in range(len(catGroup)):
        cat_lis += genrateIndexList(catGroup[i], chartIndex=15, chartCount=4)[0]
    final_lis.append(cat_lis)
else:
    final_lis.append(-1)

sec_lis = []
if sectors:
    for i in range(len(secGroup)):
        sec_lis += genrateIndexList(secGroup[i], chartIndex=15, chartCount=4)[0]
    final_lis.append(sec_lis)
else:
    final_lis.append(-1)

seg_lis=[]
if segments:
    for i in range(len(segGroup)):
        seg_lis += genrateIndexList(segGroup[i], chartIndex=15, chartCount=4)[0]
    final_lis.append(seg_lis)

else:
    final_lis.append(-1)

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup)):
        subseg_lis +=  genrateIndexList(subsegGroup[i], chartIndex=15, chartCount=4)[0]
    final_lis.append(subseg_lis)
else:
    final_lis.append([-1])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup)):
        subcat_lis +=  genrateIndexList(subcatGroup[i], chartIndex=15, chartCount=4)[0]
    final_lis.append(subcat_lis)
else:
    final_lis.append([-1])

### New Slide 1

In [55]:
# d = Sector_MonthYear['Boots']
# d = DetectHeader(d).reset_index(drop=True)
# d['Sector'] = d['Sector'].fillna(method = 'ffill')
# d = d[~d['Sector'].str.contains('Total', case=False)].reset_index(drop=True)
# d['year'] = pd.to_datetime(d['MonthYear'], format='%b-%y').dt.year
# #yearly_avg_sales = d.groupby(['year', 'Sector'])['Value Sales'].mean().reset_index(drop=True) 
# yearly_avg_sales = d.groupby(['year', 'Sector'])['Value Sales'].transform('mean').reset_index(drop=True) 

# print(yearly_avg_sales)

In [56]:
def MonthYear_clean(data,column):
    month_year_data={}
    for key,df in data.items():
        df=DetectHeader(data[key]).reset_index(drop=True)
        df[column] = df[column].fillna(method = 'ffill')
        df = df[~df[column].str.contains('Total', case=False)].reset_index(drop=True)
        df['year'] = pd.to_datetime(df['MonthYear'], format='%b-%y').dt.year
        df['year'] = df['year'].astype(int)
        yearly_avg_sales = df.groupby(['year', column])['Value Sales'].transform('mean').reset_index(drop=True) 
        yearly_avg_sales = yearly_avg_sales.replace(0, float('nan'))

        # Calculate 'Sales index' and handle NaN values gracefully
        df['Sales index'] = (df['Value Sales'] / yearly_avg_sales * 100).fillna(0).astype(int)

        # df['Sales index'] = (df['Value Sales'] / yearly_avg_sales * 100).astype(int)
        if df.shape[0]>0:
            # if column =='Sector':
            #     newkey =key
            #     month_year_data[newkey] = df
            # elif column == 'Segment':
            #     newkey= key.split(' | ')[0] + ' | ' + segments[-1]
            #     month_year_data[newkey] = df
            # elif column == 'SubSegment':
            #     newkey= key.split(' | ')[0] + ' | ' + subsegments[-1]
            #     month_year_data[newkey] = df
            # else:
            #     newkey= key.split(' | ')[0] + ' | ' + subcategories[-1]
            month_year_data[key] = df
    return month_year_data

In [57]:
Sector_MonthYear = {
    f"{key} | {categories[0]}": value for key, value in Sector_MonthYear.items()
}


In [58]:
sector_month_year = MonthYear_clean(Sector_MonthYear,column='Sector')
segment_month_year = MonthYear_clean(Segment_MonthYear,column='Segment')
subcat_month_year = MonthYear_clean(SubCategory_MonthYear,column='SubCategory')
subseg_month_year = MonthYear_clean(SubSegment_MonthYear,column='SubSegment')

In [59]:
def split_month_year(data, column):
    final_month_year ={}
    for key,df in data.items():
        for sec in df[column].unique():
            newkey = key + ' | ' + sec
            new_df = df[df[column] == sec].reset_index(drop=True)
            if new_df.shape[0] > 0:
                final_month_year[newkey] = new_df
    return final_month_year

In [60]:
sector_month_year1 = split_month_year(sector_month_year,'Sector')
segment_month_year1 = split_month_year(segment_month_year,'Segment')
subseg_month_year1 = split_month_year(subseg_month_year,'SubSegment')
subcat_month_year1 = split_month_year(subcat_month_year,'SubCategory')

In [61]:
# month_year1 = {}
# month_year1.update(sector_month_year1)
# month_year1.update(segment_month_year1)
# month_year1.update(subcat_month_year1)
# month_year1.update(subseg_month_year1)

In [62]:
if len(sectors) != 0:
    brandMarketSector = [key for key in sector_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in segment_month_year1.keys() if any(cat == key.split(' | ')[2]  for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in subseg_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in subcat_month_year1.keys() if any(cat == key.split(' | ')[2] for cat in subcategories )]

In [63]:
def completeDates1(dfList, promotionsEndOfWeekCleaned,column="Sector"):
    """
    Complete dates for each dataframe in dfList based on promotionsEndOfWeekCleaned.

    Parameters:
    dfList (list): List of dataframe keys.
    promotionsEndOfWeekCleaned (dict): Dictionary containing cleaned promotions end of week data.

    Returns:
    tuple: Tuple containing EndOfWeekcompletDate (dictionary), dfGroup (list), and dic (dictionary).
    """
    # Create a list of unique brand-category combinations
    brandCatList = sorted(set(key.split(' | ')[0]  for key in dfList))
    # Initialize dictionaries and lists
    EndOfWeekcompletDate = {}
    dfGroup = []
    dic = defaultdict(int)
    
    # Count occurrences of each brand-category combination
    for key in brandCatList:
        for name in dfList:
            if (key.split(' | ')[0] == name.split(' | ')[0]):
                
                dic[key] += 1
                
    # Iterate over unique brand-category combinations
    for name in dic.keys():

        if column == "Sector" :
            
            # Filter dataframe keys associated with the current brand-category combination
            dfName = [key for key in dfList if name == key.split(' | ')[0] and len(name.split(' | ')) == 1]
        else:
            
            dfName = [key for key in dfList if name == key.split(' | ')[0]  ]
        
        # Extract unique dates from all associated dataframes
        uniqueDates = pd.concat([promotionsEndOfWeekCleaned[key] for key in dfName])[['MonthYear']].drop_duplicates()
        # Initialize dictionary for complete dates for each dataframe
        dfCompleteDates = {}
        # Add dataframe keys to the group list
        dfGroup.append(dfName)
        # Populate EndOfWeekcompletDate dictionary with dataframes merged on unique dates
        for key in dfName:
            EndOfWeekcompletDate[key] = pd.merge(uniqueDates, promotionsEndOfWeekCleaned[key], how='left')#.replace(np.nan, 0)
            column = EndOfWeekcompletDate[key].columns[1]
            year = EndOfWeekcompletDate[key].columns[3]
            monthyear = EndOfWeekcompletDate[key].columns[0]
            EndOfWeekcompletDate[key][column] = EndOfWeekcompletDate[key][column].fillna(method='ffill')      
            EndOfWeekcompletDate[key][year] = pd.to_datetime(EndOfWeekcompletDate[key][monthyear], format='%b-%y').dt.year
            EndOfWeekcompletDate[key] = EndOfWeekcompletDate[key].fillna(0)
    return EndOfWeekcompletDate, dfGroup, dic

In [64]:
if len(sectors) != 0:
    dfSector0,secGroup0,secDuplication0=completeDates1(brandMarketSector,sector_month_year1,column="Sector")
if len(segments) != 0:
    dfSegment0,segGroup0,segDuplication0=completeDates1(brandMarketSegment,segment_month_year1,column="Segment")
if len(subsegments) != 0:
    dfSubSegment0,subsegGroup0,subsegDuplication0=completeDates1(brandMarketSubSegment,subseg_month_year1,column="Subsegment")
if len(subcategories) != 0:
    dfSubCategory0,subcatGroup0,subcatDuplication0=completeDates1(brandMarketSubCategory,subcat_month_year1,column="Subcategory")

In [65]:
def groupingkeys(data):
    grouped = defaultdict(list)

    for sublist in data:
        for entry in sublist:
            prefix = " | ".join(entry.split(" | ")[:2])  # Extract first two parts
            grouped[prefix].append(entry)
    result = list(grouped.values())
    return result

In [66]:
secGroup0=groupingkeys(secGroup0)
segGroup0=groupingkeys(segGroup0)
subsegGroup0=groupingkeys(subsegGroup0)
subcatGroup0=groupingkeys(subcatGroup0)

In [67]:
final_lis0 = []

sec_lis = []
if sectors:
    for i in range(len(secGroup0)):
        sec_lis += genrateIndexList(secGroup0[i], chartIndex=20, chartCount=6)[0]
    final_lis0.append(sec_lis)
else:
    final_lis0.append(-1)

seg_lis=[]
if segments:
    for i in range(len(segGroup0)):
        seg_lis += genrateIndexList(segGroup0[i], chartIndex=20, chartCount=6)[0]
    final_lis0.append(seg_lis)

else:
    final_lis0.append(-1)

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup0)):
        subseg_lis +=  genrateIndexList(subsegGroup0[i], chartIndex=20, chartCount=6)[0]
    final_lis0.append(subseg_lis)
else:
    final_lis0.append([-1])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup0)):
        subcat_lis +=  genrateIndexList(subcatGroup0[i], chartIndex=20, chartCount=6)[0]
    final_lis0.append(subcat_lis)
else:
    final_lis0.append([-1])

### New slide 2

In [68]:
modified_endofweek_P12M = {}
past_12_months = pd.date_range(end=end_date , periods=12, freq='M').strftime('%b-%y').tolist()
for key in modified_promotionEndOfWeek.keys():
    df=modified_promotionEndOfWeek[key].copy()
    df['End of Week'] = pd.to_datetime(df['End of Week'])
    filtered_df = df[df['End of Week'].dt.strftime('%b-%y').isin(past_12_months)]
    if filtered_df.shape[0] >0:
        modified_endofweek_P12M[key] = filtered_df 

In [69]:
brandMarket = list(set([key.split(' | ')[0]+' | '+key.split(' | ')[1] for key in modified_endofweek_P12M]))
brandMarketCategory= [key for key in modified_endofweek_P12M.keys() if any(cat in key.split(' | ')[-1] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[-1] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[-1] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[-1] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in modified_endofweek_P12M.keys() if any(cat == key.split(' | ')[-1] for cat in subcategories )]

In [70]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,modified_endofweek_P12M)
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,modified_endofweek_P12M)
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,modified_endofweek_P12M)
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,modified_endofweek_P12M)
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,modified_endofweek_P12M)

In [71]:
def promofrequencyclean(data):        
        modified_dfCategory1 = {}
        for k in data.keys():
                chart_df=data[k].copy()
                chart_df['Weekly VSOD'] = np.where((chart_df['VSOD']>.2)&(chart_df['Value Uplift (v. base) Normalized'] != ''),1,None)
                chart_df['try'] = 0
                chart_df['New Uplift'] = 0
                chart_df['try'] = np.where((chart_df['Value Uplift (v. base) Normalized']>=2),1.8,chart_df['Value Uplift (v. base) Normalized'])
                chart_df['New Uplift'] = np.where((chart_df['Weekly VSOD']==1)&(chart_df['Value Uplift (v. base) Normalized']>0.05),chart_df['try'],None)
                if not chart_df['Weekly VSOD'].isnull().all():
                        modified_dfCategory1[k]= chart_df 
        return modified_dfCategory1

In [72]:
modified_dfCategory1=promofrequencyclean(dfCategory1)
modified_dfSector1=promofrequencyclean(dfSector1)
modified_dfSegment1=promofrequencyclean(dfSegment1)
modified_dfSubSegment1=promofrequencyclean(dfSubSegment1)
modified_dfSubCategory1=promofrequencyclean(dfSubCategory1)

In [73]:
modified_dfCategory1.keys()

dict_keys(['Billie | CVS Michigan | Manual Shave Women', 'Billie | CVS Mid South | Manual Shave Women', 'Billie | CVS Midwest | Manual Shave Women', 'Billie | CVS Northeast | Manual Shave Women', 'Billie | CVS South | Manual Shave Women', 'Billie | CVS Southwest | Manual Shave Women', 'Billie | CVS West | Manual Shave Women', 'Edgewell | CVS Michigan | Manual Shave Women', 'Edgewell | CVS Mid South | Manual Shave Women', 'Edgewell | CVS Midwest | Manual Shave Women', 'Edgewell | CVS Northeast | Manual Shave Women', 'Edgewell | CVS South | Manual Shave Women', 'Edgewell | CVS Southwest | Manual Shave Women', 'Edgewell | CVS West | Manual Shave Women', 'Edgewell | Walmart North | Manual Shave Women', 'Edgewell | Walmart West | Manual Shave Women', 'Hydro Silk | CVS Michigan | Manual Shave Women', 'Hydro Silk | CVS Mid South | Manual Shave Women', 'Hydro Silk | CVS Midwest | Manual Shave Women', 'Hydro Silk | CVS Northeast | Manual Shave Women', 'Hydro Silk | CVS South | Manual Shave Wome

In [74]:
brandMarket = list(set([key.split(' | ')[0]+' | '+key.split(' | ')[1] for key in modified_endofweek_P12M]))
brandMarketCategory= [key for key in modified_dfCategory1.keys() if any(cat in key.split(' | ')[-1] for cat in categories )]
if len(sectors) != 0:
    brandMarketSector = [key for key in modified_dfSector1.keys() if any(cat == key.split(' | ')[-1] for cat in sectors )]
if len(segments) != 0:
    brandMarketSegment = [key for key in modified_dfSegment1.keys() if any(cat == key.split(' | ')[-1] for cat in segments )]
if len(subsegments) != 0:
    brandMarketSubSegment = [key for key in modified_dfSubSegment1.keys() if any(cat == key.split(' | ')[-1] for cat in subsegments )]
if len(subcategories) != 0:
    brandMarketSubCategory = [key for key in modified_dfSubCategory1.keys() if any(cat == key.split(' | ')[-1] for cat in subcategories )]

In [75]:
if len(categories) != 0:
    dfCategory1,catGroup1,catDuplication1=completeDates(brandMarketCategory,modified_endofweek_P12M)
if len(sectors) != 0:
    dfSector1,secGroup1,secDuplication1=completeDates(brandMarketSector,modified_endofweek_P12M)
if len(segments) != 0:
    dfSegment1,segGroup1,segDuplication1=completeDates(brandMarketSegment,modified_endofweek_P12M)
if len(subsegments) != 0:
    dfSubSegment1,subsegGroup1,subsegDuplication1=completeDates(brandMarketSubSegment,modified_endofweek_P12M)
if len(subcategories) != 0:
    dfSubCategory1,subcatGroup1,subcatDuplication1=completeDates(brandMarketSubCategory,modified_endofweek_P12M)

In [76]:
def group_by_region(data):
    # Define categories for grouping
    market_groups = {
        "RETAILER_REGIONS": regions_RET,
        "RETAILER_CHANNELS": channels_RET,
        "RETAILER_MARKET": market_RET,
        "CHANNEL_REGIONS": regions_CHAN,
        "CHANNEL_CHANNELS": channels_CHAN,
        "CHANNEL_MARKET": market_CHAN,
        f"{customareas}_REGIONS": regions_CUST,
        f"{customareas}_CHANNELS": channels_CUST,
        f"{customareas}_MARKET": market_CUST,
    }
    result = []
    for sublist in data:
        for category, keywords in market_groups.items():
            # Filter items matching the current category
            base_category = category.split("_")[0]

            group = [
                f"{item} | {base_category}" for item in sublist if item.split(" | ")[1] in keywords
            ]
            if group:  # Append only non-empty groups
                result.append(group)
    return result


In [77]:
catGroup1 = group_by_region(catGroup1)
secGroup1 = group_by_region(secGroup1)
segGroup1 = group_by_region(segGroup1)
subsegGroup1 = group_by_region(subsegGroup1)
subcatGroup1 = group_by_region(subcatGroup1)

In [78]:
catGroup1

[['Billie | CVS Michigan | Manual Shave Women | REGIONS',
  'Billie | CVS Mid South | Manual Shave Women | REGIONS',
  'Billie | CVS Midwest | Manual Shave Women | REGIONS',
  'Billie | CVS Northeast | Manual Shave Women | REGIONS',
  'Billie | CVS South | Manual Shave Women | REGIONS',
  'Billie | CVS Southwest | Manual Shave Women | REGIONS',
  'Billie | CVS West | Manual Shave Women | REGIONS'],
 ['Edgewell | CVS Michigan | Manual Shave Women | REGIONS',
  'Edgewell | CVS Mid South | Manual Shave Women | REGIONS',
  'Edgewell | CVS Midwest | Manual Shave Women | REGIONS',
  'Edgewell | CVS Northeast | Manual Shave Women | REGIONS',
  'Edgewell | CVS South | Manual Shave Women | REGIONS',
  'Edgewell | CVS Southwest | Manual Shave Women | REGIONS',
  'Edgewell | CVS West | Manual Shave Women | REGIONS',
  'Edgewell | Walmart North | Manual Shave Women | REGIONS',
  'Edgewell | Walmart West | Manual Shave Women | REGIONS'],
 ['Hydro Silk | CVS Michigan | Manual Shave Women | REGIONS',

In [79]:
final_lis1 = []
cat_lis = []
if categories:
    for i in range(len(catGroup1)):
        cat_lis += genrateIndexList(catGroup1[i], chartIndex=26, chartCount=4)[0]
    final_lis1.append(cat_lis)
else:
    final_lis1.append(-1)

sec_lis = []
if sectors:
    for i in range(len(secGroup1)):
        sec_lis += genrateIndexList(secGroup1[i], chartIndex=26, chartCount=4)[0]
    final_lis1.append(sec_lis)
else:
    final_lis1.append(-1)

seg_lis=[]
if segments:
    for i in range(len(segGroup1)):
        seg_lis += genrateIndexList(segGroup1[i], chartIndex=26, chartCount=4)[0]
    final_lis1.append(seg_lis)

else:
    final_lis1.append(-1)

subseg_lis =[]
if subsegments:
    for i in range(len(subsegGroup1)):
        subseg_lis +=  genrateIndexList(subsegGroup1[i], chartIndex=26, chartCount=4)[0]
    final_lis1.append(subseg_lis)
else:
    final_lis1.append([-1])

subcat_lis =[]
if subcategories:
    for i in range(len(subcatGroup1)):
        subcat_lis +=  genrateIndexList(subcatGroup1[i], chartIndex=26, chartCount=4)[0]
    final_lis1.append(subcat_lis)
else:
    final_lis1.append([-1])

In [80]:
retailer=regions_RET+channels_RET+market_RET
channels=regions_CHAN+channels_CHAN+channels_CHAN
customarea=regions_CUST+channels_CUST+market_CUST
def addarea(modified_dfCategory1,retailer,market="RETAILER"):
    keys_to_modify = [k for k in modified_dfCategory1.keys() if k.split(" | ")[1] in retailer]
    for k in keys_to_modify:
        new_key = k + " | "+ market
        modified_dfCategory1[new_key] = modified_dfCategory1[k]  
        del modified_dfCategory1[k]       
    return modified_dfCategory1              
modified_dfCategory1=addarea(modified_dfCategory1,retailer,market="RETAILER")
modified_dfCategory1=addarea(modified_dfCategory1,channels,market="CHANNELS")
modified_dfCategory1=addarea(modified_dfCategory1,customarea,market=f"{customareas}")

modified_dfSector1=addarea(modified_dfSector1,retailer,market="RETAILER")
modified_dfSector1=addarea(modified_dfSector1,channels,market="CHANNELS")
modified_dfSector1=addarea(modified_dfSector1,customarea,market=f"{customareas}")

modified_dfSegment1=addarea(modified_dfSegment1,retailer,market="RETAILER")
modified_dfSegment1=addarea(modified_dfSegment1,channels,market="CHANNELS")
modified_dfSegment1=addarea(modified_dfSegment1,customarea,market=f"{customareas}")

modified_dfSubSegment1=addarea(modified_dfSubSegment1,retailer,market="RETAILER")
modified_dfSubSegment1=addarea(modified_dfSubSegment1,channels,market="CHANNELS")
modified_dfSubSegment1=addarea(modified_dfSubSegment1,customarea,market=f"{customareas}")

modified_dfSubCategory1=addarea(modified_dfSubCategory1,retailer,market="RETAILER")
modified_dfSubCategory1=addarea(modified_dfSubCategory1,channels,market="CHANNELS")
modified_dfSubCategory1=addarea(modified_dfSubCategory1,customarea,market=f"{customareas}")


In [81]:
f"{customareas}"

'REGIONS'

In [82]:
for k in list(modified_promotionBrandsP12M.keys()):  # Convert to list to avoid runtime errors
    df = modified_promotionBrandsP12M[k].copy()
    # Filter rows based on 'Top Brands'
    df = df[~df['Top Brands'].str.contains('Others', case=False, na=False)]
    df = df[~df['Top Brands'].str.contains('Grand Total', case=False, na=False)]
    df = df[df['Value Share'] > 0.01]
    if not df.empty:
        modified_promotionBrandsP12M[k] = df
    else:
        del modified_promotionBrandsP12M[k]



## Slide duplication: index, duplication and section names

In [83]:
final_lis1

[[30, 29, 30, 30, 27, 30, 30, 27, 30, 30, 27, 30, 30, 27],
 [30,
  29,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  29],
 [30,
  29,
  30,
  29,
  30,
  30,
  27,
  28,
  30,
  30,
  27,
  30,
  29,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  29,
  30,
  30,
  27,
  28,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  29],
 [30,
  29,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  30,
  27,
  28,
  30,
  30,
  27,
  30,
  29,
  30,
  30,
  27,
  28,
  30,
  30,
  27,
  28,
  30,
  29,
  30,
  30,
  27],
 [30,
  29,
  30,
  29,
  30,
  30,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  29,
  30,
  30,
  27,
  30,
  30,
  27,
  30,
  29,
  30,
  30,
  27,
  29,
  30,
  30,
  30,
  30,
  28,
  30,
  30,
  27]]

In [84]:
index = [*[0]*5,
         *[1]*5,
         *[2]*5,
         *[3]*5,
         *[4]*5,
         *[5]*5,
         *[6]*5,
         *[7]*5,
         *[8]*5,
         *[9]*4,
         *[10]*5,
         *[11]*5,
         *[12]*5,
         *[13]*5,
         *[14]*5,
         *[15]*5,
         *final_lis,
         *[20]*5,
         *final_lis0,
         *final_lis1,
         *[0]*5,
         *[1]*5,
         *[2]*5,
         *[3]*5,
         *[10]*5,
         *[11]*5,
         *[12]*5,
         *[13]*5,
         *[14]*5
        ]
duplication = combine_duplications(Scope,count_df,[modified_promotionBrandsP12M, #0
                                                   promotionsBrandSortedTotalFinal, #1
                                                   newpromotionsNotBrandsWithMarket, #2
                                                   concated, #3
                                                   modified_promotionProductsP12M_volumeuplift, #4
                                                   new_modified_promotionProductsP12M, #5
                                                   new_modified_promotionProductsP12M, #6
                                                   top20clientonly, #7
                                                   bottom20clientonly,#8
                                                   modified_promotionBrandsP12M, #10
                                                   newModifiedBrands, #11
                                                   PromoSalesTypes_data if promo_type else None,#12
                                                   modified_promotionBrandsP12M if feature_share else None, #13
                                                   modified_promotionBrandsP12M if display_share else None, #14
                                                   modified_promotionEndOfWeek,#15
                                                   PromoRet, #16-19
                                                   modified_valueUplift, #20
                                                   #month_year1,#21
                                                   #modified_endofweek_P12M, #22
                                                   modified_promotionBrandsP12M, #0 with no client
                                                   promotionsBrandNOTSortedTotalFinal, #1 with no client
                                                   newpromotionsNotBrandsWithMarket, #2 with no client
                                                   concated, #3 with no client
                                                   modified_promotionBrandsP12M, # 10 with no client
                                                   newModifiedBrands, #11 with no client
                                                   PromoSalesTypes_data if promo_type else None,#12  with no client
                                                   modified_promotionBrandsP12M if feature_share else None, #13  with no client
                                                   modified_promotionBrandsP12M if display_share else None #14 with no client
                                                  ])
section_names = ["Promo Value Sale",#0
                 "Promo Evolution", #1
                 "VSOD Summary by Sector" , #2
                 "Value uplift by retailer by brand", #3 
                 "Volume Uplift vs discount depth",#4
                 "Value Uplift vs Promo Efficiency Quadrant", #5
                 "Top 20 promotions", #6
                 "Top 20 promotions CLIENT ONLY", #7
                 "Bottom 20 promotions CLIENT ONLY", #8
                 "Promo share vs Value Share", #10
                 "Promo Sales by total size",#11
                 "Promo Sales by promo type", #12
                 "Feature Share vs Fair Share", #13
                 "Display Share vs Fair Share", #14
                 "Promo Frequency learnings", #15
                 "Promo sales per retailer", #16-19
                 "Value Uplift vs discount depth" ,#20
                 #"Seasonality Index",#21
                 #"Promotional Frequency Analysis", #22
                 "Promo Value Sale no client prio",
                 "Promo Evolution no client prio",
                 "VSOD Summary by Sector no client prio",
                 "Value uplift by retailer by brand no client prio",
                "Promo share vs Value Share no client prio", #10
                 "Promo Sales by total size no client prio",#11
                 "Promo Sales by promo type no client prio", #12
                 "Feature Share vs Fair Share no client prio", #13
                 "Display Share vs Fair Share no client prio" #14
                ]



#duplication.insert(89, 0)

if len(sectors) > 0:
       #duplication.insert(45,(len(client_manuf)+len(client_brands))*len(categories)* len(marketList))
       duplication.insert(45, sect_vsod_count)
if len(segments) > 0:
        #duplication.insert(46,(len(client_manuf)+len(client_brands))*len(sectors)* len(marketList)) 
         duplication.insert(46, seg_vsod_count)
 
else:
    duplication.insert(46,0)  
  
if len(subsegments) > 0:
        #duplication.insert(47,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(47, subseg_vsod_count)

else:
    duplication.insert(47,0)

if len(subcategories) > 0:
        #duplication.insert(48,(len(client_manuf)+len(client_brands))*len(segments)* len(marketList))
        duplication.insert(48, subcat_vsod_count)

else:
    duplication.insert(48,0)


duplication.insert(89,1)
duplication.insert(90, 1)
duplication.insert(91, 1)
duplication.insert(92, 1)
duplication.insert(93, 1)
duplication.insert(94, 1)
duplication.insert(95, 1)
duplication.insert(96, 1)
duplication.insert(97, 1)

section_names = [f"{name} {suffix}" for name in section_names for suffix in suffixes]

section_names.insert(45,'Volume Sold on Deal Sector')
section_names.insert(46,'Volume Sold on Deal Segment')
section_names.insert(47,'Volume Sold on Deal SubSegment')
section_names.insert(48,'Volume Sold on Deal SubCategory')

section_names.insert(89,'Seasonality Index Sector')
section_names.insert(90,'Seasonality Index Segment')
section_names.insert(91,'Seasonality Index Subsegment')
section_names.insert(92,'Seasonality Index Subcategory')

section_names.insert(93,'Promotional Frequency Analysis Sector')
section_names.insert(94,'Promotional Frequency Analysis Segment')
section_names.insert(95,'Promotional Frequency Analysis Subsegment')
section_names.insert(96,'Promotional Frequency Analysis Subcategory')
section_names.insert(97,'Promotional Frequency Analysis Subcategory')

duplication[82]=1

path = os.getcwd() + '//Promotion base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated.pptx'
 

In [85]:
print(index)
print(duplication)
print(section_names)
print(len(index))
print(len(duplication))
print(len(section_names))
print(final_lis)
print(final_lis1)
print(sum(duplication))

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19], [19, 19, 19, 19, 19, 19, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 16, 19, 19, 19, 19, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 17, 19, 16, 17, 19, 19, 19, 19, 19, 19, 19, 16], [19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 16, 19, 19, 19], 20, 20, 20

In [86]:
slideDuplication(index,duplication,section_names,path,new_pre)

## If we want specific slides duplicated

In [87]:
# slides_name = ['display share', 'feature share']
# indices = [i for i, s in enumerate(section_names) if any(sub.lower() in s.lower() for sub in slides_name)]
# indices

In [88]:
# filtered_section_names = [section_names[i] for i in indices]
# filtered_duplication = [duplication[i] for i in indices]
# filtered_index = [index[i] for i in indices]

# print(filtered_section_names)
# print(filtered_duplication)
# print(filtered_index)

In [89]:
# slideDuplication(filtered_index,filtered_duplication,filtered_section_names,path,new_pre)

## Replace duplicated slides with data

In [90]:
prs = Presentation(new_pre)
posItr = 0
ind =0

In [91]:
#slide 1
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            promoValueSales(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
        ind+=1

In [92]:
print(ind,posItr)

5 197


In [93]:
# slide 2
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolution(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1


In [94]:
print(ind,posItr)

10 394


In [95]:
# slide 3
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [96]:
print(ind,posItr)

15 394


In [97]:
# slide 4
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [98]:
print(ind,posItr)

20 411


In [99]:
# slide 5
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionProductsP12M_volumeuplift,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionProductsP12M_volumeuplift.items() if key in dict[key1]}
        if filtered_dict:
            VolumeUplift(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [100]:
print(ind,posItr)

25 581


In [101]:
# slide 6
for key,value in Scope.items():
    dict = {key: count_df(new_modified_promotionProductsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in new_modified_promotionProductsP12M.items() if key in dict[key1]}
        if filtered_dict:
            ValueUpliftvsPromoEfficiencyQuadrant(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [102]:
print(ind,posItr)

30 746


In [103]:
# slide 7
for key,value in Scope.items():
    dict = {key: count_df(new_modified_promotionProductsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in new_modified_promotionProductsP12M.items() if key in dict[key1]}
        if filtered_dict:
            top20(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [104]:
print(ind,posItr)

35 911


In [105]:
# slide 8
for key,value in Scope.items():
    dict = {key: count_df(top20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in top20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            top20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [106]:
print(ind,posItr)

40 1056


In [107]:
# slide 9
for key,value in Scope.items():
    dict = {key: count_df(bottom20clientonly,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in bottom20clientonly.items() if key in dict[key1]}
        if filtered_dict:
            bot20Client(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [108]:
print(ind,posItr)

45 1203


In [109]:
if len(sectors)>0:
    newVolumeSold(prs, sect_vsod_merged, position=posItr, parent=direct_parent['Sector'], child = 'Sector')
    print(posItr)
    posItr += sect_vsod_count
    ind +=1
else:
    ind+=1

1203


In [110]:
print(ind,posItr)

46 1263


In [111]:
if len(segments)>0:
    newVolumeSold(prs, seg_vsod_merged, position=posItr, parent=direct_parent['Segment'], child = 'Segment')
    posItr += seg_vsod_count
    ind +=1
    
else:
    ind+=1

In [112]:
print(ind,posItr)

47 1383


In [113]:
if len(subsegments)>0:
    newVolumeSold(prs, subseg_vsod_merged, position=posItr, parent=direct_parent['SubSegment'], child = 'SubSegment')
    posItr += subseg_vsod_count
    ind+=1
else:
    ind+=1

In [114]:
print(ind,posItr)

48 1623


In [115]:
if len(subcategories)>0:
    newVolumeSold(prs, subcat_vsod_merged, position=posItr, parent=direct_parent['SubCategory'], child = 'SubCategory')
    posItr += subcat_vsod_count
    ind+=1
else:
    ind+=1

In [116]:
print(ind,posItr)

49 1863


In [117]:
# slide 11
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [118]:
print(ind,posItr)

54 2060


In [119]:
# slide 12
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [120]:
print(ind,posItr)

59 2060


In [121]:
# slide 13
if promo_type:
    for key,value in Scope.items():
        dict = {key: count_df(PromoSalesTypes_data,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in PromoSalesTypes_data.items() if key in dict[key1]}
            if filtered_dict:
                PromoSalesTypes(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [122]:
print(ind,posItr)

64 2060


In [123]:
# slide 14
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                featureShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [124]:
# slide 15
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                displayShare(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [125]:
print(ind,posItr)

74 2060


In [126]:
# slide 16
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionEndOfWeek,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionEndOfWeek.items() if key in dict[key1]}
        if filtered_dict:
            PromoFrequency(prs,filtered_dict,duplication[ind],position=sum(duplication[:ind]))
        posItr += len(filtered_dict)
        ind +=1

In [127]:
print(ind,posItr)

79 2753


In [128]:
if categories:
    catFinal = sorted(splitDfsPromo(dfCategory,(client_manuf) ,genrateIndexList(catGroup[0])[0]))
    catFinal = catFinal+sorted(splitDfsPromo(dfCategory,(client_brands) ,genrateIndexList(catGroup[0])[0]))
    catFinal

In [129]:
print(ind,posItr)

79 2753


In [130]:
if sectors:
    secFinal = sorted(splitDfsPromo(dfSector,(client_manuf)  ,genrateIndexList(secGroup[0])[0]))
    secFinal = secFinal + sorted(splitDfsPromo(dfSector,(client_brands)  ,genrateIndexList(secGroup[0])[0]))
    secFinal

In [131]:
print(ind,posItr)

79 2753


In [132]:
if segments:
    segFinal = sorted(splitDfsPromo(dfSegment,(client_manuf)  ,genrateIndexList(segGroup[0])[0]))
    segFinal = segFinal+sorted(splitDfsPromo(dfSegment,(client_brands)  ,genrateIndexList(segGroup[0])[0]))
    segFinal

In [133]:
if subsegments:
    subsegFinal = sorted(splitDfsPromo(dfSubSegment,(client_manuf)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal = subsegFinal + sorted(splitDfsPromo(dfSubSegment,(client_brands)  ,genrateIndexList(subsegGroup[0])[0]))
    subsegFinal

In [134]:
if subcategories:
    subcatFinal = sorted(splitDfsPromo(dfSubCategory,(client_manuf) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal = subcatFinal+sorted(splitDfsPromo(dfSubCategory,(client_brands) ,genrateIndexList(subcatGroup[0])[0]))
    subcatFinal

In [135]:
print(ind,posItr)

79 2753


In [136]:
#Slide 17
#split catGroup into Lists depends on num of charts 
catGroupSplit = splitListpromo(dfCategory, catGroup, [i-15 for i in index[ind]])

promoSalesPerRetailer(prs,dfCategory,len(index[ind]),catGroupSplit,position=sum(duplication[:ind]))
posItr = sum(duplication[:ind]) + len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split secGroup into Lists depends on num of charts 
if len(sectors) != 0: 
    secGroupSplit = splitListpromo(dfSector, secGroup, [i-15 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSector,len(index[ind]),secGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split segGroup into Lists depends on num of charts 
if len(segments) != 0: 
    segGroupSplit = splitListpromo(dfSegment, segGroup, [i-15 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSegment,len(index[ind]),segGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subsegGroup into Lists depends on num of charts 
if len(subsegments) != 0:
    subsegGroupSplit = splitListpromo(dfSubSegment, subsegGroup, [i-15 for i in index[ind]])

    promoSalesPerRetailer(prs,dfSubSegment,len(index[ind]),subsegGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1
print(len(index[ind]),ind, posItr)

#split subcatGroup into Lists depends on num of charts 
if len(subcategories) != 0:
    subcatGroupSplit = splitListpromo(dfSubCategory, subcatGroup, [i-15 for i in index[ind]])
    promoSalesPerRetailer(prs,dfSubCategory,len(index[ind]),subcatGroupSplit,position=posItr)
    posItr += len(index[ind])
ind+=1


27 80 2768
48 81 2795
44 82 2843
44 83 2887


In [137]:
# slide 21
for key,value in Scope.items():
    dict = {key: count_df(modified_valueUplift,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_valueUplift.items() if key in dict[key1]}
        if filtered_dict:
            valueUplift(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1


In [138]:
print(ind, duplication[ind], posItr)

89 1 3280


In [139]:
if len(sectors)>0:
    seasonality(prs, dfSector0, len(index[ind]), secGroup0, position=posItr,slideby="Sector")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind])) 

CVS Michigan
CVS Mid South
CVS Midwest
CVS Northeast
CVS South
CVS Southwest
CVS West
Walmart East
Walmart North
Walmart Southeast
Walmart Southwest
Walmart West
3292 24


In [140]:
if len(segments)>0:
    seasonality(prs, dfSegment0, len(index[ind]), segGroup0, position=posItr,slideby="Segment")
    posItr += len(index[ind])
ind+=1
print(posItr, len(index[ind]),ind)  

CVS Michigan
CVS Michigan
CVS Mid South
CVS Mid South
CVS Midwest
CVS Midwest
CVS Northeast
CVS Northeast
CVS South
CVS South
CVS Southwest
CVS Southwest
CVS West
CVS West
Walmart East
Walmart East
Walmart North
Walmart North
Walmart Southeast
Walmart Southeast
Walmart Southwest
Walmart Southwest
Walmart West
Walmart West
3316 52 91


In [141]:
if len(subsegments) != 0:
    seasonality(prs,dfSubSegment0,len(index[ind]),subsegGroup0,position=posItr,slideby="SubSegment")
    posItr += len(index[ind])
ind+=1

print(posItr, len(index[ind]),ind)  

CVS Michigan
CVS Michigan
CVS Michigan
CVS Michigan
CVS Mid South
CVS Mid South
CVS Mid South
CVS Mid South
CVS Midwest
CVS Midwest
CVS Midwest
CVS Midwest
CVS Northeast
CVS Northeast
CVS Northeast
CVS Northeast
CVS South
CVS South
CVS South
CVS South
CVS Southwest
CVS Southwest
CVS Southwest
CVS Southwest
CVS West
CVS West
CVS West
CVS West
Walmart East
Walmart East
Walmart East
Walmart East
Walmart East
Walmart North
Walmart North
Walmart North
Walmart North
Walmart North
Walmart Southeast
Walmart Southeast
Walmart Southeast
Walmart Southeast
Walmart Southeast
Walmart Southwest
Walmart Southwest
Walmart Southwest
Walmart Southwest
Walmart Southwest
Walmart West
Walmart West
Walmart West
Walmart West
3368 48 92


In [142]:
if len(subcategories) != 0:
    seasonality(prs,dfSubCategory0,len(index[ind]),subcatGroup0,position=posItr,slideby="SubCategory")
    posItr += len(index[ind])
ind+=1


CVS Michigan
CVS Michigan
CVS Michigan
CVS Michigan
CVS Mid South
CVS Mid South
CVS Mid South
CVS Mid South
CVS Midwest
CVS Midwest
CVS Midwest
CVS Midwest
CVS Northeast
CVS Northeast
CVS Northeast
CVS Northeast
CVS South
CVS South
CVS South
CVS South
CVS Southwest
CVS Southwest
CVS Southwest
CVS Southwest
CVS West
CVS West
CVS West
CVS West
Walmart East
Walmart East
Walmart East
Walmart East
Walmart North
Walmart North
Walmart North
Walmart North
Walmart Southeast
Walmart Southeast
Walmart Southeast
Walmart Southeast
Walmart Southwest
Walmart Southwest
Walmart Southwest
Walmart Southwest
Walmart West
Walmart West
Walmart West
Walmart West


In [143]:
print(len(index[ind]), ind, duplication[ind], posItr)

14 93 1 3416


In [144]:
print(posItr,ind)

3416 93


In [145]:
catGroup1Split = splitListpromo(modified_dfCategory1, catGroup1, [i-26 for i in index[ind]])

Promotional_Frequency(prs,modified_dfCategory1,len(index[ind]),catGroup1Split,position=posItr)
posItr +=len(catGroup1Split)
ind+=1
print(len(index[ind]), posItr)
#Sector Replace
if len(sectors) != 0: 
    secGroup1Split = splitListpromo(modified_dfSector1, secGroup1, [i-26 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSector1,len(index[ind]),secGroup1Split,position=posItr)
    posItr += len(secGroup1Split)
ind+=1
print(len(index[ind]), posItr)


if len(segments) != 0: 
    segGroup1Split = splitListpromo(modified_dfSegment1, segGroup1, [i-26 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSegment1,len(index[ind]),segGroup1Split,position=posItr)
    posItr += len(segGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subsegments) != 0:
    subsegGroup1Split = splitListpromo(modified_dfSubSegment1, subsegGroup1, [i-26 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubSegment1,len(index[ind]),subsegGroup1Split,position=posItr)
    posItr += len(subsegGroup1Split)
ind+=1
print(len(index[ind]), posItr)

if len(subcategories) != 0:
    subcatGroup1Split = splitListpromo(modified_dfSubCategory1, subcatGroup1, [i-26 for i in index[ind]])
    Promotional_Frequency(prs,modified_dfSubCategory1,len(index[ind]),subcatGroup1Split,position=posItr)
    posItr += len(subcatGroup1Split)
ind+=1


0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
25 3430
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
36 3455
0
1
2
3
0
1
2
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
33 3491
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
0
1
2
3
0
1
2
3
0
0
1
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
34 3524
0
1
2
3
0
1
2
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
1
2
0
1
2
3
0
1
2
3
0
0
1
2
0
1
2
3
0
1
2
3
0
1
2
3
0
1
2
3
0
1
0
1
2
3
0
1
2
3
0


In [146]:
#slide 1 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            promoValueSales_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
        ind+=1

In [147]:
print(ind, duplication[ind], sum(duplication[:ind]))

103 12 3313


In [148]:
# slide 2 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(promotionsBrandNOTSortedTotalFinal,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in promotionsBrandNOTSortedTotalFinal.items() if key in dict[key1]}
        if filtered_dict:
            promoEvolution(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
        ind +=1


In [149]:
# slide 3 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(newpromotionsNotBrandsWithMarket,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newpromotionsNotBrandsWithMarket.items() if key in dict[key1]}
        if filtered_dict:
            VSOD1(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [150]:
# slide 4 with no client brands
for key,value in Scope.items():
    dict = {key: count_df(concated,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in concated.items() if key in dict[key1]}
        if filtered_dict:
            valueUpliftRetailer_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [151]:
# slide 11 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(modified_promotionBrandsP12M,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
        if filtered_dict:
            PromoShare_vs_ValueShare_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [152]:
# slide 12 with no client prio
for key,value in Scope.items():
    dict = {key: count_df(newModifiedBrands,value) }
    for key1,value1 in dict.items():
        filtered_dict = {key: value for key, value in newModifiedBrands.items() if key in dict[key1]}
        if filtered_dict:
            PromoSalesTotalSize_no(prs,filtered_dict,duplication[ind],position=posItr)
        posItr += len(filtered_dict)
        ind +=1

In [153]:
# slide 13 with no client prio
if promo_type:
    for key,value in Scope.items():
        dict = {key: count_df(PromoSalesTypes_data,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in PromoSalesTypes_data.items() if key in dict[key1]}
            if filtered_dict:
                PromoSalesTypes_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [154]:
# slide 14 with no client prio
if feature_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                featureShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

In [155]:
# slide 15 with no client prio
if display_share:
    for key,value in Scope.items():
        dict = {key: count_df(modified_promotionBrandsP12M,value) }
        for key1,value1 in dict.items():
            filtered_dict = {key: value for key, value in modified_promotionBrandsP12M.items() if key in dict[key1]}
            if filtered_dict:    
                displayShare_no(prs,filtered_dict,duplication[ind],position=posItr)
            posItr += len(filtered_dict)
            ind +=1
else:
    ind +=5

## Output slides

In [156]:
outputPath=os.getcwd() + "\\Promotion EdgeWell US.pptx"
prs.save(outputPath)
# app = win32.Dispatch("PowerPoint.Application")
# presentation = app.Presentations.Open(outputPath)

## Value Uplift by product

In [157]:
%run "..\general_functions\generalFunctions.ipynb"
%run "..\Promotion Slide Duplicate\Promotion Replacement Function.ipynb"

path1 = r"/ValueUpliftvsDepth/"
loaded_data = {}
datasets_path = os.getcwd()+ path1
datasets = os.listdir(datasets_path)
for d in datasets:
    with open(datasets_path+d, 'rb') as handle:
        loaded_data[d.split('.')[0]] = pd.read_csv(handle)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'c:\\Users\\BW4SA\\Documents\\GitHub\\Slide-Automate\\Promotion Slide Duplicate/ValueUpliftvsDepth/'

In [ ]:
valueUplift_dict = {}
i=0
for key, df in loaded_data.items():
    data = DetectHeader(df)
    columns_to_ffill = [col for col in data.columns if 'item' in col.lower() or 'product' in col.lower()]
    data[columns_to_ffill] = data[columns_to_ffill].fillna(method='ffill')
    data = data[~data['Item'].str.contains('Total', case=False)].reset_index(drop=True)
    for item in data['Item'].unique():
        df = data[data['Item'] == item]
        df['Discount Depth (%)'] = df['Discount Depth (%)'].str.replace('%','').astype(float) /100
        df['Promo Price/Unit'] = df['Promo Price/Unit'].str.replace('£','').astype(float)
        if normalized:
            df['Value Uplift (v. base) Normalized'] = df['Value Uplift (v. base) Normalized'].str.replace('%','').astype(float) /100
        else:
            df['Value Uplift (v. base)'] = df['Value Uplift (v. base)'].str.replace().str.replace('%','').astype(float) /100
        df = df[df['End of Week'] != '0']
        df['End of Week'] = pd.to_datetime(df['End of Week'])
        df = df[(df['End of Week'] >= start_date) & (df['End of Week'] <= end_date)].reset_index(drop=True)
        if df.shape[0]>0 and not df['Discount Depth (%)'].isna().all():
            df = df.fillna(0).reset_index(drop = True)
            new_key = key+'_'+ item
            valueUplift_dict[new_key] = df


In [ ]:
sign = 'before'
decimals = 2
currency = '£'
data_source = "DATA SOURCE: Trade Panel/Retailer Data | Ending July 2024"


index = [20]
duplication = [len(valueUplift_dict.keys())]
section_names = ["Value Uplift by product"]
path = os.getcwd() + '//Promotion base Oct 2024.pptx'
new_pre = os.getcwd() + '//slide duplicated value.pptx'


In [ ]:
#slideDuplication(index,duplication,section_names,path,new_pre)

In [ ]:
prs = Presentation(new_pre)

In [ ]:
# Iterate through each key-slide_num pair in modified_valueUplift
for key, slide_num in zip(valueUplift_dict, range(len(valueUplift_dict.keys()))):
        # Access the slide to be modified
        slide = prs.slides[slide_num]
        
        # Extract data for the current key
        df = valueUplift_dict[key]
        #df = df[df['Value Uplift (v. base) Normalized'] !=0 ]
        # Get shapes in the slide
        shapes = slide.shapes
        
        # Find and update title shape
        titleNumber = get_shape_number(shapes, "Value Uplift vs discount depth | By Event | Category/Sector | Brand | Coop Alleanza | P12M")
        datasourcenum = get_shape_number(shapes, "Data Source | Trade Panel")
        headerNumber = get_shape_number(shapes, 'Value Uplift vs discount depth (Replace With SO WHAT)')
        if titleNumber is not None:
            shapes[datasourcenum].text = data_source
            shapes[titleNumber].text = shapes[titleNumber].text.replace('Category/Sector', key.split('_')[2]) \
                .replace('Brand | Coop Alleanza ', df['Item'][0])
            shapes[titleNumber].text_frame.paragraphs[0].font.size = Pt(12)
            shapes[titleNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'
            shapes[headerNumber].text_frame.paragraphs[0].font.size = Pt(16)
            shapes[headerNumber].text_frame.paragraphs[0].font.name = 'Nexa Bold (Headings)'

        # Create table and chart objects
        tables, charts = createTableAndChart(slide.shapes)
        chart1 = charts[0].chart  # First chart
        chart2 = charts[1].chart  # Second chart
        
        # Extract data for charts
        category = df['Item'].tolist()
        x_values_discount = df['Discount Depth (%)'].tolist()
        x_values_price = df['Promo Price/Unit'].tolist()
        if normalized:
            y_values = df['Value Uplift (v. base) Normalized'].tolist()
        else:
            y_values = df['Value Uplift (v. base)'].tolist()

        
        x_values_discount = [mround_numpy(value, 0.05) for value in x_values_discount]
        x_values_price = [mround_numpy(value, 0.5) for value in x_values_price]
        #Update first chart with Discount Depth vs Value Uplift data
        chart_data1 = XyChartData()
        series1 = chart_data1.add_series('Scatter')
        for i in range(len(category)):
            series1.add_data_point(x_values_discount[i], y_values[i])
        chart1.replace_data(chart_data1)
        
        # Access the X-axis
        
        xlsx_file = BytesIO()
        with chart_data1._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data1._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "Item")
            worksheet.write_column(1, 4, df['Item'].to_list(), None)
            worksheet.write(0, 5, "End of Week")
            worksheet.write_column(1, 5, df['End of Week'].to_list(), None)

        chart1._workbook.update_from_xlsx_blob(xlsx_file.getvalue())

        # Update second chart with Promo Price/Unit vs Value Uplift data
        chart_data2 = XyChartData()
        series2 = chart_data2.add_series('Scatter')
        for i in range(len(category)):
            series2.add_data_point(x_values_price[i], y_values[i])
        chart2.replace_data(chart_data2)
        
        x_axis = chart2.category_axis
        
        # Loop through each X-axis category label and format as currency
        if sign.lower() == 'before':
            x_axis.tick_labels.number_format = f'"{currency}"#,##0.00'  if decimals == 2 else f'"{currency}"#,##0'
        else:
            x_axis.tick_labels.number_format = f'#,##0.00"{currency}"'  if decimals == 2 else f'#,##0"{currency}"'
       
        #x_axis.has_major_gridlines = False  # Optional: remove gridlines

        xlsx_file = BytesIO()
        with chart_data2._workbook_writer._open_worksheet(xlsx_file) as (workbook, worksheet):
            chart_data2._workbook_writer._populate_worksheet(workbook, worksheet)
            worksheet.write(0, 4, "Item")
            worksheet.write_column(1, 4, df['Item'].to_list(), None)
            worksheet.write(0, 5, "End of Week")
            worksheet.write_column(1, 5, df['End of Week'].to_list(), None)
        chart2._workbook.update_from_xlsx_blob(xlsx_file.getvalue())


In [ ]:
outputPath=os.getcwd() + "\\Promotion EdgeWell ValueUplift.pptx"
prs.save(outputPath)
app = win32.Dispatch("PowerPoint.Application")
presentation = app.Presentations.Open(outputPath)